In [1]:
! git clone https://github.com/Min-KiD/video_features.git
! pip install omegaconf==2.0.6

Cloning into 'video_features'...
remote: Enumerating objects: 1302, done.
remote: Counting objects: 100% (432/432), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 1302 (delta 279), reused 281 (delta 202), pack-reused 870
Receiving objects: 100% (1302/1302), 288.07 MiB | 29.77 MiB/s, done.
Resolving deltas: 100% (699/699), done.
Updating files: 100% (25/25), done.
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
%cd video_features

/kaggle/working/video_features


In [3]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path, form_list_from_user_input
from omegaconf import OmegaConf
from tqdm import tqdm
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'Tesla T4'

In [4]:
import cv2
import math

def count_segments(video_path):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    segment_size = math.ceil(total_frames / 32)
    return segment_size

# Example usage
video_path = '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse002_x264.mp4'

Number of frames in the video: 865


In [5]:
feature_type = 'i3d'

# Load the model
extractor = ExtractI3D(args)

# Extract features
video_paths = form_list_from_user_input(args.video_paths, args.file_with_video_paths, to_shuffle=True)

print(f'The number of specified videos: {len(video_paths)}')

for video_path in tqdm(video_paths):
    segment_size = count_segments(video_path)
    
    args = OmegaConf.load(build_cfg_path(feature_type))
    args.stack_size = segment_size
    args.step_size = segment_size
    args.flow_type = 'raft'
    args.on_extraction = 'save_numpy'
    
    extractor._extract(video_path)

# for video_path in args.video_paths:
#     print(f'Extracting for {video_path}')
#     feature_dict = extractor.extract(video_path)
#     [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

The number of specified videos: 1


  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 1/1 [00:53<00:00, 53.69s/it]
